<a href="https://colab.research.google.com/github/codewithsrs/data-analysis/blob/main/car_dekho_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder

from scipy.stats import boxcox
from scipy.special import inv_boxcox

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (StandardScaler,PolynomialFeatures,MinMaxScaler,MaxAbsScaler)
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import Ridge, Lasso

from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_val_predict


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
c_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Machine learning/guided and self project/Copy of Car details v3.csv")

In [4]:
c_data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


In [5]:
c_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   owner          8128 non-null   object 
 8   mileage        7907 non-null   object 
 9   engine         7907 non-null   object 
 10  max_power      7913 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7907 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 825.6+ KB


In [6]:
col = c_data.columns

In [7]:
c_data['Company'] = c_data['name'].apply(lambda x:x.split()[0])

In [8]:
c_data.head(2)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,Company
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0,Maruti
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0,Skoda


In [9]:
p = []
for i in range (len(c_data.mileage)):
  if str(c_data.mileage[i]).endswith('kmpl'):
    p.append(str(c_data.mileage[i]).split(' ')[0])
  else:
    p.append(float(str(c_data.mileage).split()[0])*1.40)

In [10]:
c_data['Mileage'] = p

In [11]:
c_data.head(3)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,Company,Mileage
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0,Maruti,23.4
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0,Skoda,21.14
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0,Honda,17.7


In [12]:
c_data.Mileage = c_data.Mileage.astype(float)

In [13]:
c_data.Mileage.mean()

18.65321604330706

In [14]:
def m(column):
  if column < 5.00:
    return float(column)+(18.65/2)
  else:
    return column

In [15]:
c_data.Mileage = c_data.Mileage.apply(m)

In [16]:
c_data['engine'] = c_data['engine'].apply(lambda x: str(x).split()[0])

In [17]:
c_data.engine = c_data.engine.astype(float)

In [18]:
c_data.head(2)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,Company,Mileage
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248.0,74 bhp,190Nm@ 2000rpm,5.0,Maruti,23.40
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498.0,103.52 bhp,250Nm@ 1500-2500rpm,5.0,Skoda,21.14


In [19]:
c_data.max_power.unique()

array(['74 bhp', '103.52 bhp', '78 bhp', '90 bhp', '88.2 bhp',
       '81.86 bhp', '57.5 bhp', '37 bhp', '67.1 bhp', '68.1 bhp',
       '108.45 bhp', '60 bhp', '73.9 bhp', nan, '67 bhp', '82 bhp',
       '88.5 bhp', '46.3 bhp', '88.73 bhp', '64.1 bhp', '98.6 bhp',
       '88.8 bhp', '83.81 bhp', '83.1 bhp', '47.3 bhp', '73.8 bhp',
       '34.2 bhp', '35 bhp', '81.83 bhp', '40.3 bhp', '121.3 bhp',
       '138.03 bhp', '160.77 bhp', '117.3 bhp', '116.3 bhp', '83.14 bhp',
       '67.05 bhp', '168.5 bhp', '100 bhp', '120.7 bhp', '98.63 bhp',
       '175.56 bhp', '103.25 bhp', '171.5 bhp', '100.6 bhp', '174.33 bhp',
       '187.74 bhp', '170 bhp', '78.9 bhp', '88.76 bhp', '86.8 bhp',
       '108.495 bhp', '108.62 bhp', '93.7 bhp', '103.6 bhp', '98.59 bhp',
       '189 bhp', '67.04 bhp', '68.05 bhp', '58.2 bhp', '82.85 bhp',
       '81.80 bhp', '73 bhp', '120 bhp', '94.68 bhp', '160 bhp', '65 bhp',
       '155 bhp', '69.01 bhp', '126.32 bhp', '138.1 bhp', '83.8 bhp',
       '126.2 bhp', '98.

In [20]:
def spl(column):
  return str(column).split(' ')[0]

In [21]:
c_data.max_power = c_data.max_power.apply(spl)

In [22]:
c_data['max_power'].mask(c_data['max_power'] == 'nan', 0, inplace=True)

In [23]:
c_data['max_power'].mask(c_data['max_power'] == 'bhp', 0, inplace=True)

In [24]:
c_data['max_power'].mask(c_data['max_power'] == '', 0, inplace=True)

In [25]:
c_data.max_power = c_data.max_power.astype(float)

In [26]:
c_data['max_power'].mask(c_data['max_power'] == 0, 74.0, inplace=True)

In [27]:
c_data.drop('name',axis = 1,inplace = True)
c_data.drop('mileage',axis = 1,inplace = True)
c_data.head(2)

,year,selling_price,km_driven,fuel,seller_type,transmission,owner,engine,max_power,torque,seats,Company,Mileage
0,2014,450000,145500,Diesel,Individual,Manual,First Owner,1248.0,74.00,190Nm@ 2000rpm,5.0,Maruti,23.40
1,2014,370000,120000,Diesel,Individual,Manual,Second Owner,1498.0,103.52,250Nm@ 1500-2500rpm,5.0,Skoda,21.14


In [28]:
c_data.engine.unique()

array([1248., 1498., 1497., 1396., 1298., 1197., 1061.,  796., 1364.,
       1399., 1461.,  993.,   nan, 1198., 1199.,  998., 1591., 2179.,
       1368., 2982., 2494., 2143., 2477., 1462., 2755., 1968., 1798.,
       1196., 1373., 1598., 1998., 1086., 1194., 1172., 1405., 1582.,
        999., 2487., 1999., 3604., 2987., 1995., 1451., 1969., 2967.,
       2497., 1797., 1991., 2362., 1493., 1599., 1341., 1794.,  799.,
       1193., 2696., 1495., 1186., 1047., 2498., 2956., 2523., 1120.,
        624., 1496., 1984., 2354.,  814.,  793., 1799.,  936., 1956.,
       1997., 1499., 1948., 2997., 2489., 2499., 2609., 2953., 1150.,
       1994., 1388., 1527., 2199.,  995., 2993., 1586., 1390.,  909.,
       2393., 3198., 1339., 2835., 2092., 1595., 2496., 1596., 1597.,
       2596., 2148., 1299., 1590., 2231., 2694., 2200., 1795., 1896.,
       1796., 1422., 1489., 2359., 2197., 2999., 1781., 2650., 1343.,
       2446., 3498., 2198., 2776., 1950.])

In [29]:
c_data['engine'].mask(c_data['engine'] == '  nan', 0, inplace=True)

In [30]:
c_data.engine = c_data.engine.apply(lambda x:str(x).strip())

In [31]:
c_data['engine'].mask(c_data['engine'] == 'nan', 0, inplace=True)

In [32]:
c_data.engine = c_data.engine.astype(float)

In [33]:
c_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           8128 non-null   int64  
 1   selling_price  8128 non-null   int64  
 2   km_driven      8128 non-null   int64  
 3   fuel           8128 non-null   object 
 4   seller_type    8128 non-null   object 
 5   transmission   8128 non-null   object 
 6   owner          8128 non-null   object 
 7   engine         8128 non-null   float64
 8   max_power      8128 non-null   float64
 9   torque         7906 non-null   object 
 10  seats          7907 non-null   float64
 11  Company        8128 non-null   object 
 12  Mileage        8128 non-null   float64
dtypes: float64(4), int64(3), object(6)
memory usage: 825.6+ KB


In [34]:
c_data.drop(c_data[c_data.km_driven < 500].index, inplace = True)

In [35]:
c_data.owner = c_data.owner.apply(lambda x:x.split(' ')[0])
c_data.owner.unique()

array(['First', 'Second', 'Third', 'Fourth', 'Test'], dtype=object)

In [36]:
c_data['engine'].mask(c_data['engine'] == 0.0,742.0, inplace=True)

In [37]:
c_data.torque.unique()

array(['190Nm@ 2000rpm', '250Nm@ 1500-2500rpm', '12.7@ 2,700(kgm@ rpm)',
       '22.4 kgm at 1750-2750rpm', '11.5@ 4,500(kgm@ rpm)',
       '113.75nm@ 4000rpm', '7.8@ 4,500(kgm@ rpm)', '59Nm@ 2500rpm',
       '170Nm@ 1800-2400rpm', '160Nm@ 2000rpm', '248Nm@ 2250rpm',
       '78Nm@ 4500rpm', nan, '84Nm@ 3500rpm', '115Nm@ 3500-3600rpm',
       '200Nm@ 1750rpm', '62Nm@ 3000rpm', '219.7Nm@ 1500-2750rpm',
       '114Nm@ 3500rpm', '115Nm@ 4000rpm', '69Nm@ 3500rpm',
       '172.5Nm@ 1750rpm', '6.1kgm@ 3000rpm', '114.7Nm@ 4000rpm',
       '60Nm@ 3500rpm', '90Nm@ 3500rpm', '151Nm@ 4850rpm',
       '104Nm@ 4000rpm', '320Nm@ 1700-2700rpm', '250Nm@ 1750-2500rpm',
       '145Nm@ 4600rpm', '146Nm@ 4800rpm', '343Nm@ 1400-3400rpm',
       '200Nm@ 1400-3400rpm', '200Nm@ 1250-4000rpm',
       '400Nm@ 2000-2500rpm', '138Nm@ 4400rpm', '360Nm@ 1200-3400rpm',
       '200Nm@ 1200-3600rpm', '380Nm@ 1750-2500rpm', '173Nm@ 4000rpm',
       '400Nm@ 1750-3000rpm', '400Nm@ 1400-2800rpm',
       '200Nm@ 1750-3000rp

In [38]:
c_data['Torque'] = c_data.torque.apply(lambda x: str(x).split(' ')[0])

In [43]:
def splt(column):
  if str(column).endswith('Nm@'):
    return str(column)[:-3]
  elif str(column).endswith('@'):
    return str(column)[:-1]
  elif str(column).endswith('Nm'):
    return str(column)[:-2]
  elif str(column).endswith('kgm'):
    return float(str(column)[:-3])*9.80
  elif str(column).endswith('nm'):
    return str(column)[:-2]
  elif str(column).endswith('NM'):
    return str(column)[:-2]
  elif str(column).endswith('Nm(38.7kgm)'):
    return str(column)[:-11]
  elif str(column).endswith(')'):
    return str(column)[:-6]
  else:
    return column

In [44]:
c_data.Torque = c_data.Torque.apply(splt)

In [45]:
c_data['Torque'].mask(c_data['Torque'] == 'nan','40', inplace=True)

In [46]:
c_data.Torque.unique()

array(['190', '250', '12.7', '22.4', '11.5', '113.75', '7.8', '59', '170',
       '160', '248', '78', '40', '84', '115', '200', '62', '219.7', '114',
       '69', '172.5', 59.78, '114.7', '60', '90', '151', '104', '320',
       '145', '146', '343', '400', '138', '360', '380', '173', '111.7',
       '219.6', '112', '130', '205', '280', '99.04', '77', '110', '153',
       '113.7', '113', '101', '290', '120', '96', '135', '259.8', '259.9',
       '91', '96.1', '109', '202', '430', '347', '382', '620', '500',
       '550', '490', '177.5', '300', '260', '213', '224', '640', '95',
       128.38, '71', '14.9', '117', '72', '11.4', '140', '134', '150',
       '340', '240', '330', '12.5', '111.8', '11.8', '135.4', '190.25',
       '20.4', '247', '223', '180', '195', '154.9', '114.73', '108',
       '190.24', '420', '100', '51', '132', '350', '218', '24', '13.5',
       '85', '74.5', '180.4', '230', '219.66', '245', '204', '14.3',
       '125', '172', '102', '8.5', '106.5', '108.5', '144.15', '9

In [47]:
c_data.drop('torque',axis = 1,inplace = True)

In [48]:
c_data.head(2)

,year,selling_price,km_driven,fuel,seller_type,transmission,owner,engine,max_power,seats,Company,Mileage,Torque
0,2014,450000,145500,Diesel,Individual,Manual,First,1248.0,74.00,5.0,Maruti,23.40,190
1,2014,370000,120000,Diesel,Individual,Manual,Second,1498.0,103.52,5.0,Skoda,21.14,250


In [49]:
c_data.seats.unique()

array([ 5.,  4., nan,  7.,  8.,  6.,  9., 10., 14.,  2.])

In [50]:
c_data.seats = c_data.seats.apply(lambda x: str(x).strip())
c_data['seats'].mask(c_data['seats'] == 'nan','4', inplace=True)

In [51]:
c_data.seats = c_data.seats.astype(float)
c_data.isnull().sum()

year             0
selling_price    0
km_driven        0
fuel             0
seller_type      0
transmission     0
owner            0
engine           0
max_power        0
seats            0
Company          0
Mileage          0
Torque           0
dtype: int64

In [52]:
c_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8127 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   year           8127 non-null   int64  
 1   selling_price  8127 non-null   int64  
 2   km_driven      8127 non-null   int64  
 3   fuel           8127 non-null   object 
 4   seller_type    8127 non-null   object 
 5   transmission   8127 non-null   object 
 6   owner          8127 non-null   object 
 7   engine         8127 non-null   float64
 8   max_power      8127 non-null   float64
 9   seats          8127 non-null   float64
 10  Company        8127 non-null   object 
 11  Mileage        8127 non-null   float64
 12  Torque         8127 non-null   object 
dtypes: float64(4), int64(3), object(6)
memory usage: 888.9+ KB


In [53]:
c_data.Torque = c_data.Torque.astype(float)

In [54]:
ohc_cols = c_data.dtypes == object
ohc_colns = c_data.columns[ohc_cols]
le = LabelEncoder()
ohc = OneHotEncoder()
for col in ohc_colns:
    dat = le.fit_transform(c_data[col]).astype(int)
    c_data = c_data.drop(col,axis = 1)
    n_dat = ohc.fit_transform(dat.reshape(-1,1))
    n_cols = n_dat.shape[1]
    colname = ['_'.join([col,str(x)]) for x in range(n_cols)]
    new_df  = pd.DataFrame(n_dat.toarray(),index = c_data.index , columns = colname)
    c_data = pd.concat([c_data,new_df],axis = 1)

In [55]:
c_data.head(2)

,year,selling_price,km_driven,engine,max_power,seats,Mileage,Torque,fuel_0,fuel_1,fuel_2,fuel_3,seller_type_0,seller_type_1,seller_type_2,transmission_0,transmission_1,owner_0,owner_1,owner_2,owner_3,owner_4,Company_0,Company_1,Company_2,Company_3,Company_4,Company_5,Company_6,Company_7,Company_8,Company_9,Company_10,Company_11,Company_12,Company_13,Company_14,Company_15,Company_16,Company_17,Company_18,Company_19,Company_20,Company_21,Company_22,Company_23,Company_24,Company_25,Company_26,Company_27,Company_28,Company_29,Company_30,Company_31
0,2014,450000,145500,1248.0,74.00,5.0,23.40,190.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2014,370000,120000,1498.0,103.52,5.0,21.14,250.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [56]:
y_col = 'selling_price'
x_data = c_data.drop(y_col,axis = 1)
y_data = boxcox(c_data[y_col])[0]
lambda1 = boxcox(c_data[y_col])[1]

In [57]:
m = MinMaxScaler()
lr = LinearRegression()
estimator = Pipeline([("scaler", m),
                      ("regression", lr)])
kf = KFold(shuffle=True, random_state=72018, n_splits=17)
predictions = cross_val_predict(estimator, x_data, y_data, cv=kf)
r2_score(y_data, predictions)

0.9013203020365709

In [58]:
inv_boxcox(predictions,lambda1)

array([424233.39577525, 493180.22216066, 135369.01218885, ...,
       245258.81062377, 243072.04593098, 243163.17942876])

In [59]:
s = StandardScaler()
lr = LinearRegression()
estimator = Pipeline([("scaler", s),
                      ("regression", lr)])
kf = KFold(shuffle=True, random_state=72018, n_splits=17)
predictions1 = cross_val_predict(estimator, x_data, y_data, cv=kf)
r2_score(y_data, predictions1)

-2.2041322486674505e+21

In [60]:
inv_boxcox(predictions1,lambda1)

array([425756.92935691, 493049.71627928, 135241.08251059, ...,
       245224.36812855, 243076.61627357, 243142.45982441])

In [61]:
from sklearn.model_selection import GridSearchCV

# Same estimator as before
estimator = Pipeline([("scaler", MinMaxScaler()),
        ("polynomial_features", PolynomialFeatures()),
        ("ridge_regression", Ridge())])

params = {
    'polynomial_features__degree': [1,2],
    'ridge_regression__alpha': np.geomspace(0.00001, 20, 50)
}

grid = GridSearchCV(estimator, params, cv=kf)

In [62]:
grid.fit(x_data, y_data)

GridSearchCV(cv=KFold(n_splits=17, random_state=72018, shuffle=True),
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('polynomial_features',
                                        PolynomialFeatures()),
                                       ('ridge_regression', Ridge())]),
             param_grid={'polynomial_features__degree': [1, 2],
                         'ridge_regression__alpha': array([1.00000000e-05, 1.34459796e-05, 1.80794368e-05, 2.43095739e-05,
       3.26866035e-05...
       1.21960381e-02, 1.63987680e-02, 2.20497501e-02, 2.96480490e-02,
       3.98647063e-02, 5.36020029e-02, 7.20731439e-02, 9.69094024e-02,
       1.30304185e-01, 1.75206742e-01, 2.35582628e-01, 3.16763922e-01,
       4.25920124e-01, 5.72691331e-01, 7.70039597e-01, 1.03539367e+00,
       1.39218822e+00, 1.87193345e+00, 2.51699790e+00, 3.38435025e+00,
       4.55059045e+00, 6.11871465e+00, 8.22721125e+00, 1.10622915e+01,
       1.48743346e+01, 2.00

In [63]:
grid.best_score_, grid.best_params_

(0.9244782111386725,
 {'polynomial_features__degree': 2,
  'ridge_regression__alpha': 0.23558262807016522})

In [64]:
y_predict = grid.predict(x_data)

In [65]:
r2_score(y_data, y_predict)

0.9326431460888687

In [66]:
inv_boxcox(y_predict,lambda1)

array([428601.6462618 , 437176.9547466 , 158525.57287292, ...,
       257076.03157444, 254093.60152272, 254093.60152272])